# template から HTML と JS を生成する
## テスト用のデータを test-tsumugi にコピー


In [1]:
import os
from pathlib import Path
import json
import gzip
from collections import defaultdict

import pandas as pd

In [2]:
# Move up to top directory

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI-dev-branch/notebooks/notebools-web
/mnt/e/Research/TSUMUGI-dev-branch


# 古いファイルの消去


In [3]:
%%bash

rm -rf test-tsumugi/data
rm -rf test-tsumugi/app
# 10 sec

In [4]:
%%bash

mkdir -p test-tsumugi/data/phenotype
mkdir -p test-tsumugi/data/genesymbol
mkdir -p test-tsumugi/app/phenotype
mkdir -p test-tsumugi/app/genesymbol
mkdir -p test-tsumugi/app/genelist/

cp -r TSUMUGI/css test-tsumugi/ # copy top's CSS
cp -r TSUMUGI/js test-tsumugi/ # copy top's JS

cp -r TSUMUGI/template/css test-tsumugi/app/ # copy app's CSS
cp -r TSUMUGI/template/js test-tsumugi/app/ # copy app's JS


## index.html の描画に必要なファイルを生成する


In [5]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    if not Path("data", "network", "mp_term_name", f"{mp_term}.json.gz").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    mp_terms[mp_term_name_space] = mp_term

In [6]:
print(list(mp_terms)[:3])
print(len(mp_terms))  # 515

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
517


In [7]:
json.dump(mp_terms, open("data/overlap/available_mp_terms.json", "w"), indent=2)
pd.DataFrame(mp_terms.keys()).to_csv(
    "data/overlap/available_mp_terms.txt", index=False, header=False
)

In [8]:
%%bash

cp data/overlap/available_mp_terms.json test-tsumugi/data/available_mp_terms.json
cp data/overlap/available_mp_terms.txt test-tsumugi/data/available_mp_terms.txt
cp data/overlap/available_gene_symbols.txt test-tsumugi/data/available_gene_symbols.txt
cp data/annotation/symbol_mgiid.json test-tsumugi/data/marker_symbol_accession_id.json


## index.html を生成


In [9]:
%%bash

grep -v "REMOVE_THIS_LINE" TSUMUGI/template/template_index.html > test-tsumugi/index.html

# Template から HTML と JS を生成


### binariyの表現型を抽出

In [10]:
path_files = Path("TSUMUGI", "data", "phenotype").glob("*.json.gz")
binary_phenotypes = []
for path_file in path_files:
    mp_term_name = path_file.name.replace(".json.gz", " ")
    with gzip.open(path_file, "rt") as f:
        json_data: list[dict] = json.load(f)

    count_node_color = defaultdict(int)
    for data in json_data:
        if "node_color" not in data["data"]:
            continue
        count_node_color[data["data"]["node_color"]] += 1
    if len(count_node_color) == 1 and 1 in count_node_color:
        binary_phenotypes.append(mp_term_name)

print(len(binary_phenotypes))
print(binary_phenotypes[:3])
Path("data", "annotation", "binary_phenotypes.txt").write_text("\n".join(binary_phenotypes))

154
['abnormal_abdominal_wall_morphology ', 'abnormal_adrenal_gland_morphology ', 'abnormal_allantois_morphology ']


3761

In [11]:
%%bash

########################################
# Phenotype
########################################

find data/mp_term_name -type f |
    sed "s|data/mp_term_name/||" |
    sed "s|.csv$||" |
    grep -e ^male_infertility -e ^increased_fasting -e "^prewe" | # <- ここで興味のあるphenotypeを選択
    while read -r mp_term_name_underscore; do
        echo $mp_term_name_underscore

        # --------------------------
        # Data
        # --------------------------
        cp -r data/network/mp_term_name/"$mp_term_name_underscore".json.gz test-tsumugi/data/phenotype
        
        mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
        impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/annotation/mptermname_phenotypeurl.tsv)
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # --------------------------
        # HTML
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのフィルターを削除
        mode="non-binary-phenotype"
        grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt && mode="binary_phenotype"

        if [ "$mode" = "non-binary-phenotype" ] ; then

            # -----------------------------
            # contol-panel.html
            # -----------------------------
            template_file="TSUMUGI/template/html/control-panel.html"
            insert_file="TSUMUGI/template/html/control-panel-phenotype.html"
            output_file="/tmp/control-panel.html"
            replacement=$(<"$insert_file")
            # AWKで置換（プレースホルダーを一時的にマーカーに置換し、そこに挿入）
            awk -v f="$insert_file" 'BEGIN{r=""; while((getline l<f)>0) r=r l ORS; close(f)} {gsub(/XXX_PHENOTYPE/, r); print}' "$template_file" > "$output_file"


            # -----------------------------
            # cy-container.html
            # -----------------------------
            template_file="TSUMUGI/template/html/cy-container.html"
            insert_file="TSUMUGI/template/html/cy-container-phenotype.html"
            output_file="/tmp/cy-container.html"
            replacement=$(<"$insert_file")
            # AWKで置換（プレースホルダーを一時的にマーカーに置換し、そこに挿入）
            awk -v f="$insert_file" 'BEGIN{r=""; while((getline l<f)>0) r=r l ORS; close(f)} {gsub(/XXX_PHENOTYPE/, r); print}' "$template_file" > "$output_file"
        else
            sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/control-panel.html > /tmp/control-panel.html
            sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/cy-container.html > /tmp/cy-container.html
        fi

        # -----------------------------
        # head.html
        # ----------------------------- 
        template_file="TSUMUGI/template/html/head.html"
        output_file="/tmp/head.html"
        
        cat "$template_file" |
            sed "s|XXX_TITLE|${mp_term_name_space}|" |
            sed "s|XXX_JS_FILE_NAME|${mp_term_name_underscore}|" > "$output_file"

        # -----------------------------
        # header.html
        # ----------------------------- 
        template_file="TSUMUGI/template/html/header.html"
        insert_html="<a href='${impc_url}' target='_blank'>${mp_term_name_space}</a>"
        output_file="/tmp/header.html"
        
        cat "$template_file" |
            sed "s|XXX_TITLE|${insert_html}|" > "$output_file"

        # -----------------------------
        # template_app.html
        # -----------------------------
        template_file="TSUMUGI/template/html/template_app.html"
        head_file="/tmp/head.html"
        header_file="/tmp/header.html"
        control_panel_file="/tmp/control-panel.html"
        cy_container_file="/tmp/cy-container.html"
        
        escape() {
                sed -e 's/[\/&]/\\&/g' -e ':a;N;$!ba;s/\n/\\n/g' "$1"
        }
        head_html=$(escape "$head_file")
        header_html=$(escape "$header_file")
        control_panel_html=$(escape "$control_panel_file")
        cy_container_html=$(escape "$cy_container_file")

        # 置換して \n を改行に戻す
        sed -e "s|XXX_HEAD|$head_html|" \
            -e "s|XXX_H1|$header_html|" \
            -e "s|XXX_CONTROL_PANEL|$control_panel_html|" \
            -e "s|XXX_CY_CONTAINER|$cy_container_html|" \
            "$template_file" | sed 's/\\n/\
        /g' > test-tsumugi/app/phenotype/"$mp_term_name_underscore".html

        # --------------------------
        # Javascript
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのスライダーを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/template/phenotype/template_network_phenotype.js |
                sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
        else
            cat TSUMUGI/template/phenotype/template_network_phenotype.js |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
        fi

    done

increased_fasting_circulating_glucose_level


male_infertility
preweaning_lethality,_complete_penetrance
preweaning_lethality,_incomplete_penetrance


In [12]:
%%bash

########################################
# Gene Symbol
########################################

cat data/overlap/available_gene_symbols.txt |
    grep -e Rab10 -e Asxl1 -e Ddx46 -e Ap3b2 -e Kcnma1 | # <- ここで興味のあるgene symbolを選択
    while read gene_symbol; do
        echo $gene_symbol

        # --------------------------
        # Data
        # --------------------------

        cp -f data/network/gene_symbol/"$gene_symbol".json.gz test-tsumugi/data/genesymbol

        mgi_id=$(awk -F "\t" -v key="$gene_symbol" '$1 == key {print $2}' data/annotation/symbol_mgiid.tsv)
        impc_url="https://www.mousephenotype.org/data/genes/"${mgi_id}
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # HTML
        sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/control-panel.html > /tmp/control-panel.html
        sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/cy-container.html > /tmp/cy-container.html

        # -----------------------------
        # head.html
        # ----------------------------- 
        template_file="TSUMUGI/template/html/head.html"
        
        cat "$template_file" |
            sed "s|XXX_TITLE|${gene_symbol}|" |
            sed "s|XXX_JS_FILE_NAME|${gene_symbol}|" > /tmp/head.html

        # -----------------------------
        # header.html
        # ----------------------------- 
        template_file="TSUMUGI/template/html/header.html"
        insert_html="<a href='${impc_url}' target='_blank'>${gene_symbol}</a>"
        
        sed "s|XXX_TITLE|${insert_html}|" "$template_file" > /tmp/header.html

        # -----------------------------
        # template_app.html
        # -----------------------------
        template_file="TSUMUGI/template/html/template_app.html"
        head_file="/tmp/head.html"
        header_file="/tmp/header.html"
        control_panel_file="/tmp/control-panel.html"
        cy_container_file="/tmp/cy-container.html"
        
        escape() {
                sed -e 's/[\/&]/\\&/g' -e ':a;N;$!ba;s/\n/\\n/g' "$1"
        }
        head_html=$(escape "$head_file")
        header_html=$(escape "$header_file")
        control_panel_html=$(escape "$control_panel_file")
        cy_container_html=$(escape "$cy_container_file")

        # 置換して \n を改行に戻す
        sed -e "s|XXX_HEAD|$head_html|" \
            -e "s|XXX_H1|$header_html|" \
            -e "s|XXX_CONTROL_PANEL|$control_panel_html|" \
            -e "s|XXX_CY_CONTAINER|$cy_container_html|" \
            "$template_file" | sed 's/\\n/\
        /g' > test-tsumugi/app/genesymbol/"$gene_symbol".html

        # Javascript
        cat TSUMUGI/template/genesymbol/template_network_genesymbol.js |
            sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_genesymbol|${gene_symbol}|g" > test-tsumugi/app/genesymbol/"$gene_symbol".js

    done

Ap3b2
Asxl1
Ddx46
Kcnma1
Rab10


In [13]:
%%bash

########################################
# Gene List
########################################

# HTML
sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/control-panel.html > /tmp/control-panel.html
sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/cy-container.html > /tmp/cy-container.html

# -----------------------------
# head.html
# ----------------------------- 
template_file="TSUMUGI/template/html/head.html"

cat "$template_file" |
    sed "s|XXX_TITLE|Gene List|" |
    sed "s|XXX_JS_FILE_NAME|network_genelist|" > /tmp/head.html

# -----------------------------
# header.html
# ----------------------------- 
template_file="TSUMUGI/template/html/header.html"

sed "s|XXX_TITLE|gene list|" "$template_file" > /tmp/header.html

# -----------------------------
# template_app.html
# -----------------------------
template_file="TSUMUGI/template/html/template_app.html"
head_file="/tmp/head.html"
header_file="/tmp/header.html"
control_panel_file="/tmp/control-panel.html"
cy_container_file="/tmp/cy-container.html"

escape() {
        sed -e 's/[\/&]/\\&/g' -e ':a;N;$!ba;s/\n/\\n/g' "$1"
}
head_html=$(escape "$head_file")
header_html=$(escape "$header_file")
control_panel_html=$(escape "$control_panel_file")
cy_container_html=$(escape "$cy_container_file")

# 置換して \n を改行に戻す
sed -e "s|XXX_HEAD|$head_html|" \
    -e "s|XXX_H1|$header_html|" \
    -e "s|XXX_CONTROL_PANEL|$control_panel_html|" \
    -e "s|XXX_CY_CONTAINER|$cy_container_html|" \
    "$template_file" | sed 's/\\n/\
/g' > test-tsumugi/app/genelist/network_genelist.html

# Javascript
cat TSUMUGI/template/genelist/template_network_genelist.js |
    sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
    grep -v "REMOVE_THIS_LINE" > test-tsumugi/app/genelist/network_genelist.js


In [14]:
%%bash

prettier --write "test-tsumugi/index.html" --print-width 120 --prose-wrap never --tab-width 4
prettier --write "test-tsumugi/js/**/*" --print-width 120 --prose-wrap never --tab-width 4
prettier --write "test-tsumugi/app/**/*" --print-width 120 --prose-wrap never --tab-width 4


test-tsumugi/index.html 81ms (unchanged)
test-tsumugi/js/genelist_formatter.js 47ms
test-tsumugi/js/top.js 35ms
test-tsumugi/app/css/app.css 38ms (unchanged)
test-tsumugi/app/genelist/network_genelist.html 33ms
test-tsumugi/app/genelist/network_genelist.js 72ms
test-tsumugi/app/genesymbol/Ap3b2.html 10ms
test-tsumugi/app/genesymbol/Ap3b2.js 19ms
test-tsumugi/app/genesymbol/Asxl1.html 10ms
test-tsumugi/app/genesymbol/Asxl1.js 17ms
test-tsumugi/app/genesymbol/Ddx46.html 10ms
test-tsumugi/app/genesymbol/Ddx46.js 15ms
test-tsumugi/app/genesymbol/Kcnma1.html 10ms
test-tsumugi/app/genesymbol/Kcnma1.js 11ms
test-tsumugi/app/genesymbol/Rab10.html 10ms
test-tsumugi/app/genesymbol/Rab10.js 16ms
test-tsumugi/app/js/components.js 2ms (unchanged)
test-tsumugi/app/js/data_loader.js 3ms (unchanged)
test-tsumugi/app/js/exporter.js 4ms (unchanged)
test-tsumugi/app/js/filters.js 6ms (unchanged)
test-tsumugi/app/js/mobile_panel.js 3ms (unchanged)
test-tsumugi/app/js/searcher.js 5ms (unchanged)
test-tsumu

In [15]:
%%bash

uname -a # OS name
date +"%Y/%m/%d %H:%M:%S" # Last update

Linux Sycom-2021 5.15.167.4-microsoft-standard-WSL2 #1 SMP Tue Nov 5 00:21:55 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
2025/03/30 13:04:08
